<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">MODELOS DE RECOMENDACIÓN PARA EL DESARROLLO DE UN SISTEMA DE RECOMENDACIÓN DE PELÍCULAS - PROYECTO MARKETING</strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [ ]:
#Conexión de Google Colab con Drive.
from google.colab import drive
drive.mount('/content/drive')

#Acceder a los archivos de la carpeta marketing para el uso de funciones.
import sys
sys.path.append('/content/drive/MyDrive/marketing/Marketing')

Mounted at /content/drive


In [ ]:
import pandas as pd # Manipulación y el analísis de datos.
import sqlite3 as sql # Interactuar con base de datos SQL usando Python.

import matplotlib.pyplot as plt
import seaborn as sns

#### CONEXIÓN CON LA BASE DE DATOS

In [ ]:
#Establece una conexión a la base de datos SQLite ubicada en "content/db_movies"
conexion = sql.connect('/content/drive/MyDrive/marketing/Marketing/data/db_movies_copy.db')

# Crear un cursor a partir de la conexión, que permita ejecutar comando SQL sobre la base de datos.
cur = conexion.cursor()

In [ ]:
# Ejecuta una consulta SQL que obtiene los nombres de todas las tablas existentes en la base de datos.
cur.execute("SELECT name FROM sqlite_master where type='table' ")

# Recupera todos los resultados de la consulta anterior en forma de lista de tuplas.
cur.fetchall()

[('ratings',), ('movies',)]

In [ ]:
## SISTEMAS DE RECOMENDACION BASADAS EN CONTENIDO

In [ ]:
query_generos = """
SELECT movieId, titulo,
       Action, Comedy, Drama, Horror, Romance, Thriller,
       Adventure, Animation, Children, Documentary,
       Fantasy, IMAX, Musical, Mystery, War, Western, "Film-Noir"
FROM movies;
"""

peliculas_df = pd.read_sql_query(query_generos, conexion)
peliculas_df.set_index('movieId', inplace=True)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Extraer solo columnas de géneros
generos = peliculas_df.drop(columns=['titulo'])

# Calcular matriz de similitud
similitud = cosine_similarity(generos)

# Convertir a DataFrame
import numpy as np
similitud_df = pd.DataFrame(similitud, index=peliculas_df.index, columns=peliculas_df.index)


In [ ]:
def recomendar_peliculas(movie_title, peliculas_df, similitud_df, top_n=5):
    # Buscar el ID de la película
    pelicula_id = peliculas_df[peliculas_df['titulo'].str.lower() == movie_title.lower()].index

    if pelicula_id.empty:
        print("Película no encontrada.")
        return

    pelicula_id = pelicula_id[0]

    # Obtener las similitudes
    similares = similitud_df[pelicula_id].sort_values(ascending=False)

    # Excluir la misma película
    similares = similares.drop(pelicula_id)

    # Obtener títulos de las películas similares
    recomendadas = peliculas_df.loc[similares.head(top_n).index, 'titulo']

    print(f"\n🎬 Películas similares a: {peliculas_df.loc[pelicula_id, 'titulo']}\n")
    for i, titulo in enumerate(recomendadas, 1):
        print(f"{i}. {titulo}")


In [ ]:
recomendar_peliculas("Toy Story", peliculas_df, similitud_df)



🎬 Películas similares a: Toy Story

1. Adventures of Rocky and Bullwinkle, The
2. Tale of Despereaux, The
3. Asterix and the Vikings (Astérix et les Vikings)
4. The Good Dinosaur
5. Home


In [ ]:
## recoemndacion colaborativa

In [ ]:
import pandas as pd

# Leer los datos de ratings (si no están cargados ya)
ratings_query = """
SELECT * FROM ratings;
"""
ratings_df = pd.read_sql_query(ratings_query, conexion)

# Crear la matriz de usuario vs película
ratings_matrix = ratings_df.pivot_table(index='userId', columns='movieId', values='rating')

# Ver la matriz de calificaciones
ratings_matrix.head()


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Rellenar valores nulos con la media de las calificaciones de cada película
ratings_matrix_filled = ratings_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)

# Ver la matriz con valores completados
ratings_matrix_filled.head()


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,4.366379,4.000000,4.366379,4.366379,4.000000,4.366379,4.366379,4.366379,4.366379,...,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379
2,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,...,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276
3,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,...,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897
4,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,...,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556
5,4.000000,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,...,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud entre usuarios
user_similarity = cosine_similarity(ratings_matrix_filled)

# Convertir la matriz de similitudes a un DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=ratings_matrix_filled.index, columns=ratings_matrix_filled.index)

# Ver las primeras similitudes
user_similarity_df.head()


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.999540,0.998160,0.998164,0.999447,0.998588,0.998274,0.999449,0.999261,0.998669,...,0.999552,0.999010,0.994342,0.999343,0.998831,0.997378,0.999032,0.994775,0.999559,0.996020
2,0.999540,1.000000,0.998499,0.998417,0.999780,0.998974,0.998616,0.999754,0.999586,0.999020,...,0.999880,0.999354,0.994722,0.999706,0.999202,0.997699,0.999308,0.994893,0.999899,0.996356
3,0.998160,0.998499,1.000000,0.997016,0.998364,0.997615,0.997243,0.998351,0.998206,0.997639,...,0.998502,0.997956,0.993448,0.998325,0.997819,0.996182,0.997921,0.993456,0.998524,0.995045
4,0.998164,0.998417,0.997016,1.000000,0.998294,0.997567,0.997336,0.998315,0.998126,0.997690,...,0.998412,0.998032,0.993435,0.998238,0.997836,0.996321,0.997893,0.993248,0.998444,0.994881
5,0.999447,0.999780,0.998364,0.998294,1.000000,0.998887,0.998528,0.999656,0.999482,0.998889,...,0.999785,0.999269,0.994671,0.999606,0.999073,0.997622,0.999232,0.994793,0.999815,0.996228


In [ ]:
def recomendar_peliculas_collaborative(user_id, ratings_matrix, user_similarity_df, top_n=5):
    # Obtener las películas que el usuario ya ha calificado
    rated_movies = ratings_matrix.loc[user_id].dropna().index

    # Obtener usuarios más similares
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).drop(user_id)

    # Tomar los usuarios más similares
    most_similar_users = similar_users.head(top_n).index

    # Recomendaciones basadas en los usuarios más similares
    recommended_movies = []
    for user in most_similar_users:
        movies_rated_by_similar_user = ratings_matrix.loc[user].dropna().index
        for movie in movies_rated_by_similar_user:
            if movie not in rated_movies:
                recommended_movies.append(movie)

    # Obtener los títulos de las películas recomendadas
    recommended_movie_titles = peliculas_df.loc[recommended_movies, 'titulo'].dropna()

    print(f"\n🎥 Películas recomendadas para el usuario {user_id}:\n")
    for i, movie in enumerate(recommended_movie_titles.head(top_n), 1):
        print(f"{i}. {movie}")


In [ ]:
recomendar_peliculas_collaborative(6, ratings_matrix, user_similarity_df)



🎥 Películas recomendadas para el usuario 6:

1. Toy Story
2. Sudden Death
3. Four Rooms
4. Mighty Aphrodite
5. Quiz Show


In [ ]:
## sistema de recomendacion por año de estreno

In [ ]:
def recomendar_por_año(df, año, top_n=10):
    """
    Recomienda películas del año indicado, ordenadas por calificación promedio.

    Parámetros:
    - df: DataFrame con las columnas 'movieId', 'titulo', 'year', 'rating'
    - año: Año de estreno (int)
    - top_n: Número de películas recomendadas

    Retorna:
    - DataFrame con las películas recomendadas
    """
    # Filtrar por año
    df_filtrado = df[df['year'] == año]

    if df_filtrado.empty:
        print(f"No se encontraron películas del año {año}.")
        return pd.DataFrame()

    # Calcular promedio de rating por película
    recomendaciones = (
        df_filtrado
        .groupby(['movieId', 'titulo'])
        .agg(promedio_rating=('rating', 'mean'), total_votos=('rating', 'count'))
        .reset_index()
        .sort_values(by='promedio_rating', ascending=False)
    )

    return recomendaciones.head(top_n)


In [ ]:
# Recomendaciones para el año 2010
recomendar_por_año(df, 2014)


,movieId,titulo,promedio_rating,total_votos
980,110127,Noah,5.0,1
793,94810,Eva,5.0,1
781,92535,Louis C.K.: Live at the Beacon Theater,5.0,1
970,109487,Interstellar,5.0,2
909,104069,Louis C.K.: Oh My God,5.0,1
581,50872,Ratatouille,5.0,1
533,34405,Serenity,5.0,1
496,8914,Primer,5.0,1
435,6442,Belle époque,5.0,1
475,7573,Never Say Never Again,5.0,1


In [ ]:
###recomendacion ppr genero

In [ ]:
def recomendar_por_genero(df, genero, top_n=10, min_votos=10):
    """
    Recomienda películas dentro de un género específico.

    Parámetros:
    - df: DataFrame unido de ratings y movies
    - genero: string del nombre del género (por ejemplo, 'Comedy')
    - top_n: número de películas recomendadas
    - min_votos: mínimo de calificaciones para considerar la película

    Retorna:
    - DataFrame con películas recomendadas
    """
    # Verificar que el género existe en las columnas
    if genero not in df.columns:
        print(f"Género '{genero}' no encontrado.")
        return pd.DataFrame()

    # Filtrar las películas que pertenecen a ese género
    df_genero = df[df[genero] == 1]

    # Calcular promedio de rating y número de votos
    recomendaciones = (
        df_genero
        .groupby(['movieId', 'titulo'])
        .agg(promedio_rating=('rating', 'mean'), total_votos=('rating', 'count'))
        .reset_index()
    )

    # Filtrar por mínimo de votos
    recomendaciones = recomendaciones[recomendaciones['total_votos'] >= min_votos]

    # Ordenar por promedio de rating
    recomendaciones = recomendaciones.sort_values(by='promedio_rating', ascending=False)

    return recomendaciones.head(top_n)


In [ ]:
recomendar_por_genero(df, 'Action', top_n=10)


,movieId,titulo,promedio_rating,total_votos
233,1927,All Quiet on the Western Front,4.350000,10
141,1209,Once Upon a Time in the West (C'era una volta ...,4.305556,18
1110,55721,Elite Squad (Tropa de Elite),4.300000,10
1772,168252,Logan,4.280000,25
362,2959,Fight Club,4.272936,218
412,3508,"Outlaw Josey Wales, The",4.250000,18
1130,58559,"Dark Knight, The",4.238255,149
136,1197,"Princess Bride, The",4.232394,142
36,260,Star Wars: Episode IV - A New Hope,4.231076,251
373,3030,Yojimbo,4.230769,13
